### **Phase 1: Player Detection and Initial Analysis** 🏃‍♂️⚽️

1. **YOLOv11 Fine-Tuning for Player Detection** 🎯
   - Adapt YOLOv11 for detecting football players on the field.
   - **Objective**: Ensure robust detection of players, even under challenging scenarios like crowded scenes or varying lighting conditions.

In [ ]:
import torch
print(torch.backends.mps.is_available())
print(torch.cuda.is_available())

False
True


In [10]:
!yolo task=detect mode=train model="/content/drive/MyDrive/FootCVision/utils/yolo11s.pt" data="/content/drive/MyDrive/dataset/data.yaml" epochs=10 imgsz=640 device=0

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/FootCVision/utils/yolo11s.pt, data=/content/drive/MyDrive/dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

1.2 **Inference on Youtube Video** !

In [2]:
!yolo task=detect mode=predict model="/Users/alyazouzou/Desktop/CV_Football/FootCVision2/phase1/runs/detect/train/weights/best.pt" source="/Users/alyazouzou/Desktop/CV_Football/vids/demo.mov" device=mps project="/Users/alyazouzou/Desktop/CV_Football/vids" name="output"

Ultralytics 8.3.57 🚀 Python-3.9.21 torch-2.5.1 MPS (Apple M3)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs

video 1/1 (frame 1/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 17 players, 1 referee, 188.4ms
video 1/1 (frame 2/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 17 players, 1 referee, 26.0ms
video 1/1 (frame 3/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 14.1ms
video 1/1 (frame 4/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 15.7ms
video 1/1 (frame 5/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 14.6ms
video 1/1 (frame 6/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo.mov: 384x640 1 goalkeeper, 18 players, 1 referee, 19.0ms
video 1/1 (frame 7/3229) /Users/alyazouzou/Desktop/CV_Football/vids/demo

2. **Conformal Object Detection with puncc library** 📏

In [20]:
import os
import glob
import numpy as np
from inference import PlayerInference

from deel.puncc.object_detection import SplitBoxWise
from deel.puncc.metrics import object_detection_mean_coverage, object_detection_mean_area
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [21]:
dataset_path = "/Users/alyazouzou/Desktop/CV_Football/dataset"
train_images_path = os.path.join(dataset_path, "train/images")
train_labels_path = os.path.join(dataset_path, "train/labels")

def parse_yolo_dataset(images_path, labels_path, percentage=0.01, random_seed=42):
    """
    Parse only a subset of the YOLO dataset (e.g., 5%) and extract image paths and ground truth boxes.
    Args:
        images_path (str): Path to the images folder.
        labels_path (str): Path to the labels folder.
        percentage (float): Percentage of data to parse (e.g., 0.05 for 5%).
        random_seed (int): Random seed for reproducibility.
    Returns:
        Tuple[List[str], List[np.ndarray]]: Subsampled image paths and ground truth boxes.
    """
    # Use glob to get all .txt files
    label_files = glob.glob(os.path.join(labels_path, "*.txt"))

    # Set random seed for reproducibility
    np.random.seed(random_seed)

    # Determine the number of files to sample
    total_files = len(label_files)
    sample_size = int(percentage * total_files)

    # Randomly select a subset of files
    sampled_files = np.random.choice(label_files, size=sample_size, replace=False)

    image_paths = []
    ground_truth_boxes = []

    for label_file in sampled_files:
        with open(label_file, "r") as f:
            labels = f.readlines()

        # Extract bounding boxes (YOLO format: class_id x_center y_center width height)
        boxes = []
        for label in labels:
            parts = list(map(float, label.strip().split()))
            _, x_center, y_center, width, height = parts
            # Convert YOLO format to bounding box (x1, y1, x2, y2)
            x1 = x_center - (width / 2)
            y1 = y_center - (height / 2)
            x2 = x_center + (width / 2)
            y2 = y_center + (height / 2)
            boxes.append([x1, y1, x2, y2])

        # Add the image path and boxes
        image_file = os.path.join(images_path, os.path.basename(label_file).replace(".txt", ".jpg"))
        image_paths.append(image_file)
        ground_truth_boxes.append(boxes)

    return image_paths, ground_truth_boxes


# Parse the YOLO dataset
image_paths, ground_truth_boxes = parse_yolo_dataset(train_images_path, train_labels_path)

# Convert to numpy arrays for compatibility
X = np.array(image_paths)
y = np.array(ground_truth_boxes, dtype=object)  # Use dtype=object for variable-length arrays

In [22]:
print(y)

[list([[0.08671875, 0.3765625, 0.10703125000000001, 0.509375], [0.1859375, 0.32109375, 0.20781249999999998, 0.43671875], [0.15234375, 0.55234375, 0.19453125, 0.69296875], [0.21484375, 0.27265625, 0.23828125, 0.36640625], [0.28515625, 0.31484375, 0.31328124999999996, 0.41640625], [0.38125000000000003, 0.24531250000000002, 0.4015625, 0.34218750000000003], [0.484375, 0.425, 0.509375, 0.5484375], [0.42421875000000003, 0.48749999999999993, 0.45234375, 0.615625], [0.56484375, 0.55703125, 0.5929687499999999, 0.6867187499999999], [0.6281249999999999, 0.32578125, 0.6546875, 0.41171875], [0.82421875, 0.2421875, 0.84765625, 0.3375], [0.43671875000000004, 0.19296875000000002, 0.45703125, 0.28515625], [0.35234375, 0.1640625, 0.37734375000000003, 0.253125], [0.5867187500000001, 0.390625, 0.59921875, 0.4140625]])
 list([[0.4609375, 0.496875, 0.47187500000000004, 0.5453125], [0.77890625, 0.82890625, 0.80703125, 0.89453125], [0.33046875, 0.26015625000000003, 0.33828125, 0.28984375], [0.415625, 0.68125,

In [23]:
# Split data into train, calibration, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_fit, X_calib, y_fit, y_calib = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [24]:
#initialize player inference class
player_inference = PlayerInference()

In [25]:
# Perform conformal prediction
conformal_predictor = SplitBoxWise(predictor=player_inference, method="multiplicative", train=False)

conformal_predictor.fit(
    X_fit=X_fit,
    y_fit=y_fit,
    X_calib=X_calib,
    y_calib=y_calib
)

# Predict conformal intervals
alpha = 0.2
y_pred, box_inner, box_outer = conformal_predictor.predict(X_test=image_paths, alpha=alpha)

RuntimeError: Provided model has no predict method. Use a BasePredictor or a DualPredictor to build a compatible predictor.

In [ ]:
# Evaluate the results
coverage = object_detection_mean_coverage(box_outer, y_test)
average_area = object_detection_mean_area(box_outer)

print(f"Marginal coverage: {np.round(coverage, 2)}")
print(f"Average area of prediction intervals: {np.round(average_area, 2)}")